<a href="https://colab.research.google.com/github/MughalUsama/Brain-tumor-segmentation-FYP/blob/d3/resnettest_fyp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
import numpy as np
from matplotlib import pyplot as plt
import h5py
import cv2
import tensorflow as tf

In [3]:
import os
def create_model():
    input_t = tf.keras.Input(shape=(512,512,3))
    res_model = tf.keras.applications.ResNet50(include_top=False,input_tensor=input_t)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Lambda(lambda image: tf.image.resize(image, (512, 512)))) 
    model.add(res_model)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    model.build((None,512,512,3))
    print(model.summary())
    model.compile(loss='categorical_crossentropy',
                  optimizer="rmsprop",
                  metrics=['accuracy'])
    return model


In [4]:
checkpoint_path = "drive/MyDrive/cpoint.ckpt"

model = create_model()
model.load_weights(checkpoint_path)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 512, 512, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 16, 16, 2048)      23587712  
_________________________________________________________________
flatten (Flatten)            (None, 524288)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               134217984 
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 771       
Total params: 157,807,491
Trainable params: 157,753,859
Non-trainable params: 53,632
_____________________________________

In [5]:
# Xtrain = np.ndarray(shape = (900,512,512)) #1564 + 500 + 1000
# Ytrain = np.zeros(900)

# Xval = np.ndarray(shape = (500,512,512)) #1564 + 500 + 1000
# Yval = np.zeros(500)

Xtest = np.ndarray(shape = (500,512,512)) #1564 + 500 + 1000
Ytest = np.zeros(500)

train_count = 0
test_count= 0
val_count = 0
i = 2
while i <3065:   
    # if train_count < 900:
    #     if i < 767:
    #       path = '/content/drive/MyDrive/TUMOR DATASET/tumor1.zip (Unzipped Files)/'
    #     elif i < 1533:
    #       path = '/content/drive/MyDrive/TUMOR DATASET/tumor3.zip (Unzipped Files)/'
    #     elif i < 2299:
    #       path = '/content/drive/MyDrive/TUMOR DATASET/tumor2.zip (Unzipped Files)/'
    #     else:
    #       path = '/content/drive/MyDrive/TUMOR DATASET/tumor4.zip (Unzipped Files)/'

    #     f = h5py.File(path+str(i)+'.mat','r')
    #     gr = f.get('cjdata')
    #     im = np.array(gr.get('image'))
    #     norm_image = cv2.normalize(im, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F) #for normalizing the images of dataset
    #     norm_image = cv2.resize(norm_image, (512,512), interpolation = cv2.INTER_AREA)
    #     Xtrain[train_count] = np.array(norm_image)
    #     Ytrain[train_count] = gr.get('label')[()]
    #     train_count +=1
    #     i+=3
    #     print(i)

    if test_count < 500:
        if i < 767:
          path = '/content/drive/MyDrive/TUMOR DATASET/tumor1.zip (Unzipped Files)/'
        elif i < 1533:
          path = '/content/drive/MyDrive/TUMOR DATASET/tumor3.zip (Unzipped Files)/'
        elif i < 2299:
          path = '/content/drive/MyDrive/TUMOR DATASET/tumor2.zip (Unzipped Files)/'
        else:
          path = '/content/drive/MyDrive/TUMOR DATASET/tumor4.zip (Unzipped Files)/'

        f = h5py.File(path+str(i)+'.mat','r')
        gr = f.get('cjdata')
        im = np.array(gr.get('image'))
        norm_image = cv2.normalize(im, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F) #for normalizing the images of dataset
        norm_image = cv2.resize(norm_image, (512,512), interpolation = cv2.INTER_AREA)

        Xtest[test_count] = np.array(norm_image)
        Ytest[test_count] = gr.get('label')[()]
        test_count +=1
        i+=3
        print(i)
    else:
      break
    # if val_count < 500:
    #     if i < 767:
    #       path = '/content/drive/MyDrive/TUMOR DATASET/tumor1.zip (Unzipped Files)/'
    #     elif i < 1533:
    #       path = '/content/drive/MyDrive/TUMOR DATASET/tumor3.zip (Unzipped Files)/'
    #     elif i < 2299:
    #       path = '/content/drive/MyDrive/TUMOR DATASET/tumor2.zip (Unzipped Files)/'
    #     else:
    #       path = '/content/drive/MyDrive/TUMOR DATASET/tumor4.zip (Unzipped Files)/'

    #     f = h5py.File(path+str(i)+'.mat','r')
    #     gr = f.get('cjdata')
    #     im = np.array(gr.get('image'))
    #     norm_image = cv2.normalize(im, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F) #for normalizing the images of dataset
    #     norm_image = cv2.resize(norm_image, (512,512), interpolation = cv2.INTER_AREA)

    #     Xval[val_count] = np.array(norm_image)
    #     Yval[val_count] = gr.get('label')[()]
    #     val_count +=1
    #     i+=1

    # print(i)

#    print(np.max(norm_image))
#    plt.imshow(norm_image, cmap="gray") 
#  plt.show()
#    print((gr.get('label')[()])) #cjdata.label: 1 for meningioma, 2 for glioma, 3 for pituitary tumor

5
8
11
14
17
20
23
26
29
32
35
38
41
44
47
50
53
56
59
62
65
68
71
74
77
80
83
86
89
92
95
98
101
104
107
110
113
116
119
122
125
128
131
134
137
140
143
146
149
152
155
158
161
164
167
170
173
176
179
182
185
188
191
194
197
200
203
206
209
212
215
218
221
224
227
230
233
236
239
242
245
248
251
254
257
260
263
266
269
272
275
278
281
284
287
290
293
296
299
302
305
308
311
314
317
320
323
326
329
332
335
338
341
344
347
350
353
356
359
362
365
368
371
374
377
380
383
386
389
392
395
398
401
404
407
410
413
416
419
422
425
428
431
434
437
440
443
446
449
452
455
458
461
464
467
470
473
476
479
482
485
488
491
494
497
500
503
506
509
512
515
518
521
524
527
530
533
536
539
542
545
548
551
554
557
560
563
566
569
572
575
578
581
584
587
590
593
596
599
602
605
608
611
614
617
620
623
626
629
632
635
638
641
644
647
650
653
656
659
662
665
668
671
674
677
680
683
686
689
692
695
698
701
704
707
710
713
716
719
722
725
728
731
734
737
740
743
746
749
752
755
758
761
764
767
770
773
776
77

In [6]:
Xtest = Xtest.reshape(Xtest.shape[0],512,512,1)
Xtest = np.repeat(Xtest[...], 3, -1)

In [7]:
Ytest = Ytest-1
loss, acc = model.evaluate(Xtest, tf.keras.utils.to_categorical(Ytest), verbose=2)


16/16 - 18s - loss: 0.6202 - accuracy: 0.8440
